In [2]:
# Test Code to GPU
import tensorflow as tf
with tf.device('/gpu:0'):
    a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
    b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
    c = tf.matmul(a, b)

with tf.Session() as sess:
    print (sess.run(c))

[[ 22.  28.]
 [ 49.  64.]]


In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
from sklearn.preprocessing import Normalizer
X = np.load("mawi_features.pkl")
Y = np.load("mawi_labels.pkl")
C = np.load("mawi_labels.pkl")
T = np.load("mawi_features.pkl")

scaler = Normalizer().fit(X)
trainX = scaler.transform(X)

scaler = Normalizer().fit(T)
testT = scaler.transform(T)

y_train = np.array(Y)
#y_test = np.array(C)


# reshape input to be [samples, time steps, features]
X_train = np.reshape(trainX, [-1,15,1])
#X_test = np.reshape(testT, [-1,15,1])

[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 8561588457990699372
, name: "/gpu:0"
device_type: "GPU"
memory_limit: 3282324684
locality {
  bus_id: 1
}
incarnation: 2853643666522367967
physical_device_desc: "device: 0, name: GeForce GTX 950M, pci bus id: 0000:01:00.0"
]


In [2]:
def conv1D_Net1(input_1d):
    kernel_1d1 = tf.truncated_normal([3,1,64], dtype=tf.float32)
    conv1d = tf.nn.conv1d(input_1d, kernel_1d1, stride=1, padding='SAME')
    conv1d = tf.nn.relu(conv1d)
    pool1 = tf.nn.pool(conv1d, [2], 'MAX', 'SAME', strides = [2])
     #Flattenning the output of ConvNets 
    flat_conv2 = tf.contrib.layers.flatten(pool1)

    datasize = flat_conv2.get_shape().as_list()[1]
    W_fc1 = tf.Variable(tf.truncated_normal([datasize, 128], stddev=0.02))
    b_fc1 = tf.Variable(tf.zeros([128]))
    h_fc1 = tf.nn.relu(tf.matmul(flat_conv2,W_fc1)+b_fc1)
    h_fc1 = tf.nn.dropout(h_fc1, keep_prob)

    # Output layer
    y_conv = tf.layers.dense(inputs=h_fc1, units=2)
    y_conv = tf.nn.sigmoid(y_conv)
    return y_conv
    
def batches(batch_size, features, labels):
    assert len(features) == len(labels)
    out_batches = []
    sample_size = len(features)
    for start_i in range(0, sample_size, batch_size):
        end_i = start_i + batch_size
        batches = [features[start_i:end_i], labels[start_i:end_i]] 
        out_batches.append(batches)
    return out_batches


In [ ]:
n_classes = 2
batchSize =  36 
# Features and Labels
features = tf.placeholder(tf.float32, [None, trainX.shape[1],1])
labels = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(dtype=tf.float32, name = "keep_prob")

# Learning rate 
learning_rate = 0.5

# Output of the model 
output = conv1D_Net1(features)

# Calculating cost
cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = output, labels = labels))
# Optimizer
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

correct_prediction = tf.equal(tf.argmax(output,1), tf.argmax(labels,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# Train or test with this batch size

epochs = 10


init = tf.global_variables_initializer() 

with tf.Session() as sess:
    sess.run(init) 
    train_batches = batches(batchSize, X_train, y_train)
    
    for epoch in range(epochs): 
        for batch_features, batch_labels in train_batches: 
            train_data = {features: X_train, labels : y_train, keep_prob : 0.7}
            training_accuracy = sess.run(accuracy, feed_dict = train_data )
            # Print status for every 10 epochs
            cost_val = sess.run(cost, feed_dict = {features: X_train, labels : y_train, keep_prob : 1} )
        if epoch % 2 == 0:
            print('Epoch {:<3} - Training Accuracy: {} loss {}'.format(epoch,training_accuracy, cost_val))

    # Save the variables to disk.
    save_model_path = './image_classification'
    # Save Model
    saver = tf.train.Saver()
    save_path = saver.save(sess, save_model_path)
    print("Model saved")